In [2]:
import numpy as np
import traceback
from PIL import Image
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
similar_images = [
    r'F:\temp\thesisdata\saatchi_micro_resized512\animal\8397090_8397090_1449107_7461448-TBLYLYPE-7.jpg',
    r'F:\temp\thesisdata\saatchi_micro_resized512\animal\8397119_8397119_1449107_7461477-LPWVXSTK-7.jpg',
    r'F:\temp\thesisdata\saatchi_micro_resized512\animal\8397160_8397160_1449107_7461515-ZYYKAMXO-7.jpg'
]

identical_images = [
    r'F:\temp\thesisdata\saatchi_micro_resized512\bike\3959361_3959361_814506_3029216-DCMXIZGI-7.jpg',
    r'F:\temp\thesisdata\saatchi_micro_resized512\bike\3959361_3959361_814506_3029216-DCMXIZGI-7.jpg'
]

very_different_images = [
    r'F:\temp\thesisdata\saatchi_micro_resized512\architecture\7792694_7792694_351147_6860670-RNGLBJHV-7.jpg',
    r'F:\temp\thesisdata\saatchi_micro_resized512\landscape\2688391_2688391_56334_1758284-JBMBGFFX-7.jpg'
]

In [11]:
def load_image(input_path: str):
    img = None
    try:
        img = Image.open(input_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
    except (OSError, IOError):
        print(f'Opening image failed: \n {traceback.format_exc()}')
    return np.array(img)


In [94]:
def MMD(x, y, kernel='multiscale'):
    """Emprical maximum mean discrepancy. The lower the result
       the more evidence that distributions are the same.

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as "multiscale" or "rbf"
    """
    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))

    dxx = rx.t() + rx - 2. * xx # Used for A in (1)
    dyy = ry.t() + ry - 2. * yy # Used for B in (1)
    dxy = rx.t() + ry - 2. * zz # Used for C in (1)

    XX, YY, XY = (torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device))

    if kernel == "multiscale":

        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
            XX += a**2 * (a**2 + dxx)**-1
            YY += a**2 * (a**2 + dyy)**-1
            XY += a**2 * (a**2 + dxy)**-1

    if kernel == "rbf":

        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
            XX += torch.exp(-0.5*dxx/a)
            YY += torch.exp(-0.5*dyy/a)
            XY += torch.exp(-0.5*dxy/a)

    return torch.mean(XX + YY - 2. * XY)

In [117]:

def load_image_bw(input_path: str):
    img = None
    try:
        img = Image.open(input_path)
        if img.mode != 'L':
            img = img.convert('L')
    except (OSError, IOError):
        print(f'Opening image failed: \n {traceback.format_exc()}')
    return np.array(img)

img0 = load_image_bw(similar_images[1])
img1 = load_image_bw(similar_images[2])
X = torch.as_tensor(torch.as_tensor(img0))
y = torch.as_tensor(torch.as_tensor(img1))

img2 = load_image(very_different_images[0])
img3 = load_image(very_different_images[0])
X = torch.as_tensor(torch.as_tensor(img2))
y = torch.as_tensor(torch.as_tensor(img3))

MMD(X, y, kernel='multiscale').item()

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
sim = [lo]

# Meerdere afbeeldingen
set1 = torch.as_tensor(np.array(()))